In [0]:
#!pip install -U pandas

In [0]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [0]:
%%time
data = pd.read_excel('/content/drive/My Drive/Data/Cleaned_transactions_data.xlsx', converters={'Cardnum':str})

CPU times: user 8.07 s, sys: 128 ms, total: 8.2 s
Wall time: 12.2 s


In [0]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


In [0]:
data = data.merge(data.groupby('Date')['Recnum'].first().rename('Recnum_start_day').reset_index().drop_duplicates(), on = 'Date', how = 'left')
data['Recnum_start_day'] = data['Recnum'] - data['Recnum_start_day']
data['Date'] = pd.to_timedelta(data['Recnum_start_day'], unit = 's') + data['Date']
data = data.drop(columns = ['Recnum_start_day'])

In [0]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01 00:00:00,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2010-01-01 00:00:01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0
2,3,5142131721,2010-01-01 00:00:02,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2010-01-01 00:00:03,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2010-01-01 00:00:04,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


In [0]:
%%time
for keys in [['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    data['_'.join(keys)] = data[keys[0]].apply(str) + '_' + data[keys[1]].apply(str)

CPU times: user 137 ms, sys: 17.1 ms, total: 154 ms
Wall time: 155 ms


In [0]:
60*60*24 

86400

In [0]:
%%time
attributes = ['Cardnum', 'Merchnum', 'Cardnum_Merchnum', 'Cardnum_Merch zip', 'Cardnum_Merch state']
days = ['86400s', '172800s', '345600s', '691200s', '1296000s', '2678400s']
fns = ['mean', 'max', 'median','sum']
for attribute in attributes:
    attribute_groupby = data.set_index('Date').groupby(attribute)
    for day in tqdm(days):
        day_text = str(int(int(day[:-1])/86400)-1)+'d'
        attribute_groupby_amt_day = attribute_groupby.rolling(day)['Amount']
        for fn in fns:    
            data['Amount_'+attribute+'_'+fn+'_'+day_text] = data[['Date', attribute]].merge(attribute_groupby_amt_day.agg(fn).reset_index(), on = ['Date', attribute], how = 'left')['Amount']
            data['Actual_Amount/'+attribute+'_'+fn+'_'+day_text] = data['Amount'] / data['Amount_'+attribute+'_'+fn+'_'+day_text]
        data[attribute+'_count_'+day_text] = data[['Date', attribute]].merge(attribute_groupby_amt_day.count().reset_index(), on = ['Date', attribute])['Amount']
    data['Days_since_'+attribute] = (data.groupby(attribute)['Date'].diff() / np.timedelta64(1, 'D')).fillna(0)

100%|██████████| 6/6 [05:22<00:00, 53.75s/it]


CPU times: user 35min 5s, sys: 3.93 s, total: 35min 9s
Wall time: 35min 12s


In [0]:
data.shape

(96397, 288)

In [0]:
%%time
for attribute in attributes[:2]:
    for day1 in tqdm(days[:2]):
        day1_text = str(int(int(day1[:-1])/86400)-1)+'d'
        for day2 in days[-3:]:
            day2_text = str(int(int(day2[:-1])/86400)-1)+'d'
            for fn1 in ['mean', 'count']:
                for fn2 in ['mean', 'count']:
                    numerator = data[[attribute, 'Date']].merge(data.set_index('Date').groupby(attribute).rolling(day1)['Amount'].agg(fn1).reset_index(), on = [attribute, 'Date'])['Amount']
                    denominator = data[[attribute, 'Date']].merge(data.set_index('Date').groupby(attribute).rolling(day2)['Amount'].agg(fn2).reset_index(level = 0).groupby(attribute).rolling('86400s')['Amount'].mean().reset_index(), on = [attribute, 'Date'])['Amount']
                    data[fn1+'_'+attribute+'_'+day1_text+'/'+'mean_'+fn2+'_'+attribute+'_'+day2_text] = numerator / denominator

100%|██████████| 2/2 [10:56<00:00, 328.02s/it]

CPU times: user 12min 23s, sys: 862 ms, total: 12min 24s
Wall time: 12min 25s


In [0]:
data.shape

(96397, 336)

In [0]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Cardnum_Merchnum,Cardnum_Merch zip,Cardnum_Merch state,Amount_Cardnum_mean_0d,Actual_Amount/Cardnum_mean_0d,Amount_Cardnum_max_0d,Actual_Amount/Cardnum_max_0d,Amount_Cardnum_median_0d,Actual_Amount/Cardnum_median_0d,Amount_Cardnum_sum_0d,Actual_Amount/Cardnum_sum_0d,Cardnum_count_0d,Amount_Cardnum_mean_1d,Actual_Amount/Cardnum_mean_1d,Amount_Cardnum_max_1d,Actual_Amount/Cardnum_max_1d,Amount_Cardnum_median_1d,Actual_Amount/Cardnum_median_1d,Amount_Cardnum_sum_1d,Actual_Amount/Cardnum_sum_1d,Cardnum_count_1d,Amount_Cardnum_mean_3d,Actual_Amount/Cardnum_mean_3d,Amount_Cardnum_max_3d,Actual_Amount/Cardnum_max_3d,Amount_Cardnum_median_3d,Actual_Amount/Cardnum_median_3d,Amount_Cardnum_sum_3d,Actual_Amount/Cardnum_sum_3d,Cardnum_count_3d,Amount_Cardnum_mean_7d,Actual_Amount/Cardnum_mean_7d,Amount_Cardnum_max_7d,Actual_Amount/Cardnum_max_7d,Amount_Cardnum_median_7d,Actual_Amount/Cardnum_median_7d,Amount_Cardnum_sum_7d,Actual_Amount/Cardnum_sum_7d,Cardnum_count_7d,Amount_Cardnum_mean_14d,Actual_Amount/Cardnum_mean_14d,Amount_Cardnum_max_14d,Actual_Amount/Cardnum_max_14d,Amount_Cardnum_median_14d,Actual_Amount/Cardnum_median_14d,Amount_Cardnum_sum_14d,Actual_Amount/Cardnum_sum_14d,Cardnum_count_14d,Amount_Cardnum_mean_30d,Actual_Amount/Cardnum_mean_30d,Amount_Cardnum_max_30d,Actual_Amount/Cardnum_max_30d,Amount_Cardnum_median_30d,Actual_Amount/Cardnum_median_30d,Amount_Cardnum_sum_30d,Actual_Amount/Cardnum_sum_30d,Cardnum_count_30d,Days_since_Cardnum,Amount_Merchnum_mean_0d,Actual_Amount/Merchnum_mean_0d,Amount_Merchnum_max_0d,Actual_Amount/Merchnum_max_0d,Amount_Merchnum_median_0d,Actual_Amount/Merchnum_median_0d,Amount_Merchnum_sum_0d,Actual_Amount/Merchnum_sum_0d,Merchnum_count_0d,Amount_Merchnum_mean_1d,Actual_Amount/Merchnum_mean_1d,Amount_Merchnum_max_1d,Actual_Amount/Merchnum_max_1d,Amount_Merchnum_median_1d,Actual_Amount/Merchnum_median_1d,Amount_Merchnum_sum_1d,Actual_Amount/Merchnum_sum_1d,Merchnum_count_1d,Amount_Merchnum_mean_3d,Actual_Amount/Merchnum_mean_3d,Amount_Merchnum_max_3d,Actual_Amount/Merchnum_max_3d,Amount_Merchnum_median_3d,Actual_Amount/Merchnum_median_3d,Amount_Merchnum_sum_3d,Actual_Amount/Merchnum_sum_3d,Merchnum_count_3d,Amount_Merchnum_mean_7d,Actual_Amount/Merchnum_mean_7d,Amount_Merchnum_max_7d,Actual_Amount/Merchnum_max_7d,Amount_Merchnum_median_7d,Actual_Amount/Merchnum_median_7d,Amount_Merchnum_sum_7d,Actual_Amount/Merchnum_sum_7d,Merchnum_count_7d,Amount_Merchnum_mean_14d,Actual_Amount/Merchnum_mean_14d,Amount_Merchnum_max_14d,Actual_Amount/Merchnum_max_14d,Amount_Merchnum_median_14d,Actual_Amount/Merchnum_median_14d,Amount_Merchnum_sum_14d,Actual_Amount/Merchnum_sum_14d,Merchnum_count_14d,Amount_Merchnum_mean_30d,Actual_Amount/Merchnum_mean_30d,Amount_Merchnum_max_30d,Actual_Amount/Merchnum_max_30d,Amount_Merchnum_median_30d,Actual_Amount/Merchnum_median_30d,Amount_Merchnum_sum_30d,Actual_Amount/Merchnum_sum_30d,Merchnum_count_30d,Days_since_Merchnum,Amount_Cardnum_Merchnum_mean_0d,Actual_Amount/Cardnum_Merchnum_mean_0d,Amount_Cardnum_Merchnum_max_0d,Actual_Amount/Cardnum_Merchnum_max_0d,Amount_Cardnum_Merchnum_median_0d,Actual_Amount/Cardnum_Merchnum_median_0d,Amount_Cardnum_Merchnum_sum_0d,Actual_Amount/Cardnum_Merchnum_sum_0d,Cardnum_Merchnum_count_0d,Amount_Cardnum_Merchnum_mean_1d,Actual_Amount/Cardnum_Merchnum_mean_1d,Amount_Cardnum_Merchnum_max_1d,Actual_Amount/Cardnum_Merchnum_max_1d,Amount_Cardnum_Merchnum_median_1d,Actual_Amount/Cardnum_Merchnum_median_1d,Amount_Cardnum_Merchnum_sum_1d,Actual_Amount/Cardnum_Merchnum_sum_1d,Cardnum_Merchnum_count_1d,Amount_Cardnum_Merchnum_mean_3d,Actual_Amount/Cardnum_Merchnum_mean_3d,Amount_Cardnum_Merchnum_max_3d,Actual_Amount/Cardnum_Merchnum_max_3d,Amount_Cardnum_Merchnum_median_3d,Actual_Amount/Cardnum_Merchnum_median_3d,Amount_Cardnum_Merchnum_sum_3d,Actual_Amount/Cardnum_Merchnum_sum_3d,Cardnum_Merchnum_count_3d,Amount_Cardnum_Merchnum_mean_7d,Actual_Amount

In [0]:
data.shape

(96397, 336)

In [0]:
len(data['Cardnum_Merchnum'].unique())

39413

In [0]:
data.isna().sum()[data.isna().sum() > 0]

Series([], dtype: int64)

In [0]:
data.to_csv('features.csv', index = False)

In [0]:
data.to_csv('/content/drive/My Drive/Data/features.csv', index = False)